# Tags Domain Namespace Testing

This notebook tests the new Tags Domain Namespace API implementation.
It demonstrates the cleaner API surface for tag management and project assignment operations.

In [ ]:
# Setup and imports
import os
import sys

sys.path.insert(0, os.path.join(os.getcwd(), "../src"))

from kili.client import Kili

In [ ]:
# Initialize Kili client with test credentials
API_KEY = ""
ENDPOINT = "http://localhost:4001/api/label/v2/graphql"

kili = Kili(
    api_key=API_KEY,
    api_endpoint=ENDPOINT,
    legacy=False,  # Use the new domain API
)

print("Kili client initialized successfully!")
print(f"Tags namespace available: {hasattr(kili, 'tags_ns')}")

## Test Tags Domain Namespace Access

In [ ]:
# Access the tags namespace
tags = kili.tags_ns
print(f"Tags namespace type: {type(tags)}")
print(f"Available methods: {[method for method in dir(tags) if not method.startswith('_')]}")

## Test Tag Listing

In [ ]:
try:
    # Test list method - all organization tags
    org_tags = tags.list()
    print(f"Organization tags: {org_tags}")
    print(f"Number of organization tags: {len(org_tags)}")

    # Test list method - project-specific tags
    project_tags = tags.list(project_id="test_project_id")
    print(f"\nProject tags: {project_tags}")
    print(f"Number of project tags: {len(project_tags)}")

    # Test list method with specific fields
    tags_specific_fields = tags.list(fields=["id", "label", "color"])
    print(f"\nTags with specific fields: {tags_specific_fields}")

except Exception as e:
    print(f"Expected error (test environment): {e}")
    print("This is normal in a test environment without tag data")

## Test Tag Creation

In [ ]:
try:
    # Test tag creation with default color
    created_tag1 = tags.create(name="notebook_test_tag")
    print(f"Created tag (default color): {created_tag1}")

    # Test tag creation with specific color
    created_tag2 = tags.create(
        name="important_notebook_tag",
        color="#ff0000",  # Red color
    )
    print(f"Created tag (red color): {created_tag2}")

    # Test tag creation with another color
    created_tag3 = tags.create(
        name="reviewed_notebook_tag",
        color="#00ff00",  # Green color
    )
    print(f"Created tag (green color): {created_tag3}")

except Exception as e:
    print(f"Expected error (test environment): {e}")
    print("This is normal in a test environment - tag creation requires organization permissions")

## Test Tag Updates

In [ ]:
try:
    # Test tag update by name
    updated_tag1 = tags.update(tag_name="notebook_test_tag", new_name="updated_notebook_tag")
    print(f"Updated tag by name: {updated_tag1}")

    # Test tag update by ID (more precise when multiple tags have same name)
    updated_tag2 = tags.update(
        tag_id="test_tag_id_123",  # Replace with actual tag ID
        new_name="precisely_updated_tag",
    )
    print(f"Updated tag by ID: {updated_tag2}")

except Exception as e:
    print(f"Expected error (test environment): {e}")
    print("This is normal - requires existing tags and organization permissions")

## Test Tag Assignment to Projects

In [ ]:
try:
    # Test assigning tags by name
    assigned_tags1 = tags.assign(
        project_id="test_project_id", tags=["important_notebook_tag", "reviewed_notebook_tag"]
    )
    print(f"Assigned tags by name: {assigned_tags1}")

    # Test assigning tags by ID
    assigned_tags2 = tags.assign(
        project_id="test_project_id",
        tag_ids=["tag_id_1", "tag_id_2"],  # Replace with actual tag IDs
    )
    print(f"Assigned tags by ID: {assigned_tags2}")

    # Test assigning single tag
    assigned_tags3 = tags.assign(project_id="test_project_id", tags=["notebook_test_tag"])
    print(f"Assigned single tag: {assigned_tags3}")

except Exception as e:
    print(f"Expected error (test environment): {e}")
    print("This is normal - requires valid project and tag IDs")

## Test Tag Unassignment from Projects

In [ ]:
try:
    # Test unassigning specific tags by name
    unassigned_tags1 = tags.unassign(project_id="test_project_id", tags=["important_notebook_tag"])
    print(f"Unassigned tags by name: {unassigned_tags1}")

    # Test unassigning specific tags by ID
    unassigned_tags2 = tags.unassign(
        project_id="test_project_id",
        tag_ids=["tag_id_1"],  # Replace with actual tag ID
    )
    print(f"Unassigned tags by ID: {unassigned_tags2}")

    # Test unassigning all tags from project
    unassigned_tags3 = tags.unassign(project_id="test_project_id", all=True)
    print(f"Unassigned all tags: {unassigned_tags3}")

except Exception as e:
    print(f"Expected error (test environment): {e}")
    print("This is normal - requires valid project with assigned tags")

## Test Tag Deletion

In [ ]:
try:
    # Test tag deletion by name
    deleted1 = tags.delete(tag_name="notebook_test_tag")
    print(f"Deleted tag by name: {deleted1}")

    # Test tag deletion by ID (more precise)
    deleted2 = tags.delete(tag_id="test_tag_id_123")  # Replace with actual tag ID
    print(f"Deleted tag by ID: {deleted2}")

    # Test deleting tag that was assigned to projects
    deleted3 = tags.delete(tag_name="important_notebook_tag")
    print(f"Deleted tag (was assigned to projects): {deleted3}")

except Exception as e:
    print(f"Expected error (test environment): {e}")
    print("This is normal - requires existing tags and organization permissions")

## Test Error Handling and Validation

In [ ]:
# Test various error conditions and validation
print("=== Testing Error Handling and Validation ===")

validation_tests = [
    {
        "name": "Update without tag_name or tag_id",
        "operation": lambda: tags.update(new_name="new_name"),
        "should_fail": True,
        "expected_error": "ValueError",
    },
    {
        "name": "Delete without tag_name or tag_id",
        "operation": lambda: tags.delete(),
        "should_fail": True,
        "expected_error": "ValueError",
    },
    {
        "name": "Assign without tags or tag_ids",
        "operation": lambda: tags.assign(project_id="test_project"),
        "should_fail": True,
        "expected_error": "ValueError",
    },
    {
        "name": "Unassign without any parameters",
        "operation": lambda: tags.unassign(project_id="test_project"),
        "should_fail": True,
        "expected_error": "ValueError",
    },
    {
        "name": "Unassign with multiple conflicting parameters",
        "operation": lambda: tags.unassign(
            project_id="test_project", tags=["tag1"], tag_ids=["id1"], all=True
        ),
        "should_fail": True,
        "expected_error": "ValueError",
    },
]

for test in validation_tests:
    try:
        print(f"\nTesting: {test['name']}")
        result = test["operation"]()
        if test["should_fail"]:
            print(f"✗ Should have failed but succeeded: {result}")
        else:
            print(f"✓ Operation succeeded as expected: {result}")
    except Exception as e:
        if test["should_fail"]:
            print(f"✓ Validation correctly failed: {type(e).__name__}: {e}")
        else:
            print(f"✗ Unexpected error: {type(e).__name__}: {e}")

## Test Tag Workflow Example

In [ ]:
# Demonstrate a complete tag workflow
print("=== Complete Tag Workflow Example ===")

try:
    # Step 1: List existing organization tags
    print("\n1. Listing existing organization tags...")
    existing_tags = tags.list()
    print(f"   Found {len(existing_tags)} existing tags")

    # Step 2: Create new tags for workflow
    print("\n2. Creating new tags for workflow...")
    workflow_tags = [
        {"name": "workflow_priority_high", "color": "#ff0000"},
        {"name": "workflow_priority_medium", "color": "#ffff00"},
        {"name": "workflow_priority_low", "color": "#00ff00"},
        {"name": "workflow_status_review", "color": "#0000ff"},
    ]

    created_tag_ids = []
    for tag_info in workflow_tags:
        try:
            created = tags.create(**tag_info)
            created_tag_ids.append(created["id"])
            print(f"   Created: {tag_info['name']} (ID: {created['id']})")
        except Exception as e:
            print(f"   Failed to create {tag_info['name']}: {e}")

    # Step 3: Assign tags to a project
    print("\n3. Assigning tags to project...")
    test_project_id = "workflow_test_project"
    try:
        assigned = tags.assign(
            project_id=test_project_id, tags=["workflow_priority_high", "workflow_status_review"]
        )
        print(f"   Assigned tags: {assigned}")
    except Exception as e:
        print(f"   Assignment failed: {e}")

    # Step 4: List project-specific tags
    print("\n4. Listing project-specific tags...")
    try:
        project_tags = tags.list(project_id=test_project_id)
        print(f"   Project has {len(project_tags)} tags assigned")
    except Exception as e:
        print(f"   Failed to list project tags: {e}")

    # Step 5: Update a tag
    print("\n5. Updating a tag...")
    try:
        updated = tags.update(
            tag_name="workflow_priority_medium", new_name="workflow_priority_normal"
        )
        print(f"   Updated tag: {updated}")
    except Exception as e:
        print(f"   Update failed: {e}")

    # Step 6: Remove some tags from project
    print("\n6. Removing tags from project...")
    try:
        unassigned = tags.unassign(project_id=test_project_id, tags=["workflow_priority_high"])
        print(f"   Unassigned tags: {unassigned}")
    except Exception as e:
        print(f"   Unassignment failed: {e}")

    # Step 7: Clean up - delete workflow tags
    print("\n7. Cleaning up workflow tags...")
    cleanup_tags = [
        "workflow_priority_high",
        "workflow_priority_normal",
        "workflow_priority_low",
        "workflow_status_review",
    ]
    for tag_name in cleanup_tags:
        try:
            deleted = tags.delete(tag_name=tag_name)
            print(f"   Deleted: {tag_name} (Success: {deleted})")
        except Exception as e:
            print(f"   Failed to delete {tag_name}: {e}")

except Exception as e:
    print(f"Workflow failed: {e}")
    print("This is expected in a test environment")

## API Comparison: Legacy vs Domain Namespace

In [ ]:
print("=== API Comparison: Legacy vs Domain Namespace ===")
print()
print("LEGACY API (legacy=True):")
print("  kili.tags_of_organization()  # List organization tags")
print("  kili.tags_of_project(project_id='proj123')  # List project tags")
print("  kili.create_tag(label='important', color='#ff0000')")
print("  kili.update_tag(tag_id='tag123', new_label='updated')")
print("  kili.delete_tag(tag_id='tag123')")
print("  kili.tag_project(project_id='proj123', tag_ids=['tag1', 'tag2'])")
print("  kili.untag_project(project_id='proj123', tag_ids=['tag1'])")
print()
print("NEW DOMAIN API (legacy=False):")
print("  kili.tags_ns.list()  # List organization tags")
print("  kili.tags_ns.list(project_id='proj123')  # List project tags")
print("  kili.tags_ns.create(name='important', color='#ff0000')")
print("  kili.tags_ns.update(tag_name='old_name', new_name='updated')")
print("  kili.tags_ns.update(tag_id='tag123', new_name='updated')")
print("  kili.tags_ns.delete(tag_name='unwanted')")
print("  kili.tags_ns.assign(project_id='proj123', tags=['tag1', 'tag2'])")
print("  kili.tags_ns.assign(project_id='proj123', tag_ids=['id1', 'id2'])")
print("  kili.tags_ns.unassign(project_id='proj123', tags=['tag1'])")
print("  kili.tags_ns.unassign(project_id='proj123', all=True)")
print()
print("Benefits of Domain Namespace API:")
print("✓ Cleaner, more intuitive method names (assign/unassign vs tag_project/untag_project)")
print("✓ More flexible parameter options (by name or ID for most operations)")
print("✓ Better validation with descriptive error messages")
print("✓ Consistent parameter naming (tag_name, new_name, project_id)")
print("✓ Enhanced IDE support with namespace autocomplete")
print("✓ Type safety with full annotations and runtime checking")
print("✓ Unified interface for organization and project tag operations")
print("✓ Support for removing all tags from project with all=True")
print("✓ More intuitive workflow for tag lifecycle management")

## Summary

This notebook demonstrates the Tags Domain Namespace implementation:

1. **Cleaner API Surface**: Methods are logically grouped under `kili.tags_ns` (when legacy=False)
2. **Intuitive Method Names**: `assign`/`unassign` instead of `tag_project`/`untag_project`
3. **Flexible Operations**: Support for operations by tag name or ID for precision
4. **Enhanced Validation**: Comprehensive parameter validation with descriptive errors
5. **Type Safety**: Full type annotations with runtime type checking
6. **Unified Interface**: Single namespace for both organization and project tag operations
7. **Better Workflow**: More intuitive tag lifecycle from creation to assignment to deletion
8. **Comprehensive Operations**: Support for bulk operations and flexible unassignment options
9. **Color Management**: Enhanced tag creation with color customization

The implementation successfully provides a more intuitive and powerful interface for tag management operations while maintaining full backward compatibility through the existing legacy methods.